In [1]:
# SPARK_HOME="/Users/ssv/spark-2.2.0-bin-hadoop2.7"
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.2.0 pyspark-shell'
os.environ["PYSPARK_PYTHON"]='python3'
# os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["SPARK_HOME"]='/Users/ssv/spark-2.2.0-bin-hadoop2.7'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))
os.environ["PYSPARK_PYTHON"] = 'python3'
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.2.0
      /_/

Using Python version 3.6.2 (default, Sep 21 2017 18:29:43)
SparkSession available as 'spark'.


In [9]:
ratings_file = 'data/u.data'
films_file = 'data/u.item'

In [20]:
ratings_rdd = sc.textFile(ratings_file)
films_rdd = sc.textFile(films_file)

In [24]:
r_df = ratings_rdd.map(lambda x: x.split('\t')).toDF(schema=['user_id', 'film_id', 'rating', 'hz'])

In [25]:
r_df.show()

+-------+-------+------+---------+
|user_id|film_id|rating|       hz|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
|    298|    474|     4|884182806|
|    115|    265|     2|881171488|
|    253|    465|     5|891628467|
|    305|    451|     3|886324817|
|      6|     86|     3|883603013|
|     62|    257|     2|879372434|
|    286|   1014|     5|879781125|
|    200|    222|     5|876042340|
|    210|     40|     3|891035994|
|    224|     29|     3|888104457|
|    303|    785|     3|879485318|
|    122|    387|     5|879270459|
|    194|    274|     2|879539794|
|    291|   1042|     4|874834944|
|    234|   1184|     2|892079237|
+-------+-------+------+---------+
only showing top 20 rows



In [27]:
f_df = films_rdd.map((lambda x: (x.split('|')[0],  x.split('|')[1]))).toDF(schema=['film_id', 'film_name'])

In [28]:
f_df.show()

+-------+--------------------+
|film_id|           film_name|
+-------+--------------------+
|      1|    Toy Story (1995)|
|      2|    GoldenEye (1995)|
|      3|   Four Rooms (1995)|
|      4|   Get Shorty (1995)|
|      5|      Copycat (1995)|
|      6|Shanghai Triad (Y...|
|      7|Twelve Monkeys (1...|
|      8|         Babe (1995)|
|      9|Dead Man Walking ...|
|     10|  Richard III (1995)|
|     11|Seven (Se7en) (1995)|
|     12|Usual Suspects, T...|
|     13|Mighty Aphrodite ...|
|     14|  Postino, Il (1994)|
|     15|Mr. Holland's Opu...|
|     16|French Twist (Gaz...|
|     17|From Dusk Till Da...|
|     18|White Balloon, Th...|
|     19|Antonia's Line (1...|
|     20|Angels and Insect...|
+-------+--------------------+
only showing top 20 rows



In [32]:
r_df.registerTempTable("ratings")
f_df.registerTempTable("films")

In [156]:
spark.sql("SHOW TABLES").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |    films|       true|
|        |  ratings|       true|
|        |       rf|       true|
|        |user_user|       true|
+--------+---------+-----------+



In [61]:
users_count = spark.sql("SELECT count(distinct(user_id)) as u_count FROM ratings").take(1)[0].u_count

In [63]:
films_count = spark.sql("SELECT count(distinct(film_id)) as f_count FROM films").take(1)[0].f_count

In [66]:
print('films count = {}, users_count = {}'.format(films_count, users_count))

films count = 1682, users_count = 943


In [79]:
average_user_ratings = round(spark.sql("SELECT count(rating) as ratings_count FROM ratings") \
                                    .take(1)[0].ratings_count / users_count, 4)

In [85]:
average_film_ratings = round(spark.sql("SELECT count(rating) as ratings_count FROM ratings") \
                                    .take(1)[0].ratings_count / films_count, 4)

In [88]:
completness = round(spark.sql("SELECT count(rating) as ratings_count FROM ratings") \
                                    .take(1)[0].ratings_count / (users_count * films_count), 4)

In [89]:
print('average_film_ratings = {}, average_user_ratings = {}, completness = {}'\
                                  .format(average_film_ratings, average_user_ratings, completness))

average_film_ratings = 59.453, average_user_ratings = 106.0445, completness = 0.063


In [108]:
user_user = spark.sql('SELECT user_id as user_id, sum(rating) as ratings_sum, count(rating) as ratings_count,' +
                       ' ROUND (sum(rating) / count(rating), 4) as users_mean_rating FROM ratings GROUP BY user_id')

In [109]:
user_user.show()

+-------+-----------+-------------+-----------------+
|user_id|ratings_sum|ratings_count|users_mean_rating|
+-------+-----------+-------------+-----------------+
|    296|      614.0|          147|           4.1769|
|    467|      162.0|           44|           3.6818|
|    691|      135.0|           32|           4.2188|
|    675|      126.0|           34|           3.7059|
|    829|      227.0|           64|           3.5469|
|    125|      626.0|          182|           3.4396|
|    451|      268.0|           98|           2.7347|
|    800|      105.0|           28|             3.75|
|    853|      122.0|           41|           2.9756|
|    666|      898.0|          245|           3.6653|
|    870|      929.0|          269|           3.4535|
|    919|      753.0|          217|             3.47|
|    926|       66.0|           20|              3.3|
|      7|     1598.0|          403|           3.9653|
|    124|       84.0|           24|              3.5|
|     51|       82.0|       

## user id for ratings prediction:  94

In [127]:
films_for_user_94 = spark.sql('SELECT film_id FROM ratings WHERE user_id = {}'.format('94'))

In [211]:
films_for_user_94.count()

400

In [147]:
spark.sql('SELECT DISTINCT(user_id) FROM ratings' + 
          ' WHERE film_id in (SELECT film_id FROM ratings WHERE user_id = {}) ORDER BY user_id'.format('94')).show()

+-------+
|user_id|
+-------+
|      1|
|     10|
|    100|
|    101|
|    102|
|    103|
|    104|
|    105|
|    106|
|    107|
|    108|
|    109|
|     11|
|    110|
|    111|
|    112|
|    113|
|    114|
|    115|
|    116|
+-------+
only showing top 20 rows



In [143]:
user_user.registerTempTable("user_user")

In [154]:
rf = spark.sql('SELECT * FROM user_user WHERE user_id in (SELECT DISTINCT(user_id) FROM ratings' + 
          ' WHERE film_id in (SELECT film_id FROM ratings WHERE user_id = {}))'.format('94'))

In [155]:
rf.registerTempTable("rf")

In [157]:
rf.show()

+-------+-----------+-------------+-----------------+
|user_id|ratings_sum|ratings_count|users_mean_rating|
+-------+-----------+-------------+-----------------+
|    296|      614.0|          147|           4.1769|
|    467|      162.0|           44|           3.6818|
|    675|      126.0|           34|           3.7059|
|    691|      135.0|           32|           4.2188|
|    829|      227.0|           64|           3.5469|
|    125|      626.0|          182|           3.4396|
|    451|      268.0|           98|           2.7347|
|    800|      105.0|           28|             3.75|
|    853|      122.0|           41|           2.9756|
|    666|      898.0|          245|           3.6653|
|    870|      929.0|          269|           3.4535|
|    919|      753.0|          217|             3.47|
|    926|       66.0|           20|              3.3|
|    124|       84.0|           24|              3.5|
|    447|      500.0|          139|           3.5971|
|     51|       82.0|       

In [353]:
def ss(x):
    print(x)
    return x

In [356]:
def s(x):
    # x - user_id
    # Calculate corellation ...
    
    my_users_mean_rating = spark.sql('SELECT users_mean_rating FROM user_user WHERE user_id = {}'.format('94')).take(1)[0].users_mean_rating
    curr_users_mean_rating = spark.sql('SELECT users_mean_rating FROM user_user WHERE user_id = {}'.format(x)).take(1)[0].users_mean_rating

    corr_s_a_u = spark.sql(
        'SELECT ' +   
        ' (sum(myR_POW_uR_) / (sqrt(sum(myR_2)) * sqrt(sum(uR_2))))  * (CASE WHEN count(T3.user_id)/50 < 1 THEN count(T3.user_id)/50 ELSE 1 END) as corr_s_a_u, '    
        ' CASE WHEN sum(myR_2) = 0 OR sum(uR_2) = 0 THEN 0 ELSE sum(myR_POW_uR_) / (sqrt(sum(myR_2)) * sqrt(sum(uR_2))) END as s_a_u ' +
        ' FROM (' +
        ' SELECT T2.user_id as user_id, T1.rating as rating_94, T2.rating as rating_u, T1.film_id,' + 
        ' int(T1.rating) - {} as myR_,  int(T2.rating) - {} as uR_,'.format(my_users_mean_rating, curr_users_mean_rating) +
        ' (int(T1.rating) - {}) * (int(T2.rating) - {}) as myR_POW_uR_,'.format(my_users_mean_rating, curr_users_mean_rating) +    
        ' POW(int(T1.rating) - {}, 2) as myR_2,'.format(my_users_mean_rating) + 
        ' POW(int(T2.rating) - {}, 2) as uR_2'.format(curr_users_mean_rating) +    
        ' FROM ' +
        '(SELECT user_id, rating, film_id FROM ratings WHERE user_id = 94) T1, ' +
        '(SELECT user_id, rating, film_id FROM ratings WHERE user_id = {}) T2 '.format(str(x)) +
        ' WHERE T1.film_id = T2.film_id' +
            ') T3 GROUP BY user_id' 
        ).take(1)[0].corr_s_a_u
        
    return float(corr_s_a_u)

In [342]:
from pyspark.sql.types import FloatType
spark.udf.register('s', s, FloatType())

Traceback (most recent call last):
  File "/Users/ssv/spark-2.2.0-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 148, in dump
    return Pickler.dump(self, obj)
  File "/Users/ssv/anaconda3/lib/python3.6/pickle.py", line 409, in dump
    self.save(obj)
  File "/Users/ssv/anaconda3/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/Users/ssv/anaconda3/lib/python3.6/pickle.py", line 736, in save_tuple
    save(element)
  File "/Users/ssv/anaconda3/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/Users/ssv/spark-2.2.0-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 249, in save_function
    self.save_function_tuple(obj)
  File "/Users/ssv/spark-2.2.0-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 297, in save_function_tuple
    save(f_globals)
  File "/Users/ssv/anaconda3/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with

PicklingError: Could not serialize object: Exception: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.

In [343]:
s('94')

0.9844289288401369

In [371]:
user_id_list = spark.sql('SELECT user_id FROM rf ORDER BY user_id').collect()

In [379]:
from tqdm import tqdm_notebook
sau = {}
for item in tqdm_notebook(user_id_list):    
    sau[item.user_id] = s(item.user_id)

A Jupyter Widget

In [380]:
import pickle
with open('sau.pickle', 'wb') as file:
    pickle.dump(sau, file)   

In [438]:
import operator
ngb_30 = sorted(sau.items(), key=operator.itemgetter(1), reverse=True)[2:32]

In [439]:
ngb_30

[('323', 0.6330644560265287),
 ('360', 0.6207357680071715),
 ('293', 0.590014743266137),
 ('267', 0.5839495927398461),
 ('249', 0.5812900892305374),
 ('854', 0.5551874664570252),
 ('916', 0.5502086675974763),
 ('624', 0.5471691239900619),
 ('773', 0.5401587418779192),
 ('560', 0.5400385495822057),
 ('592', 0.5384405626586564),
 ('595', 0.5339939852125214),
 ('269', 0.5297107916409709),
 ('343', 0.5287790427859199),
 ('880', 0.5199594422786038),
 ('591', 0.5140159694936672),
 ('389', 0.5044095713822253),
 ('339', 0.5031718536035155),
 ('292', 0.5012135911240411),
 ('643', 0.49978723662136165),
 ('933', 0.4997816379083301),
 ('135', 0.4978019565804982),
 ('537', 0.4938318949847064),
 ('457', 0.4915256070820755),
 ('747', 0.4903884269601028),
 ('645', 0.489983972124755),
 ('49', 0.4857552798705937),
 ('115', 0.47789791024935635),
 ('303', 0.47718396646562716),
 ('92', 0.4770852620351513)]

In [440]:
ngb_30_df = spark.createDataFrame(ngb_30, schema=['user_id', 'sau'])

In [441]:
ngb_30_df.registerTempTable('ngb_30')

In [ ]:
for i in (spark.sql('SELECT user_id FROM ngb_30 ORDER by sau, int(user_id)').collect()):
    print(i.user_id, ',')

In [443]:
ngb_30_df.show()

+-------+-------------------+
|user_id|                sau|
+-------+-------------------+
|    323| 0.6330644560265287|
|    360| 0.6207357680071715|
|    293|  0.590014743266137|
|    267| 0.5839495927398461|
|    249| 0.5812900892305374|
|    854| 0.5551874664570252|
|    916| 0.5502086675974763|
|    624| 0.5471691239900619|
|    773| 0.5401587418779192|
|    560| 0.5400385495822057|
|    592| 0.5384405626586564|
|    595| 0.5339939852125214|
|    269| 0.5297107916409709|
|    343| 0.5287790427859199|
|    880| 0.5199594422786038|
|    591| 0.5140159694936672|
|    389| 0.5044095713822253|
|    339| 0.5031718536035155|
|    292| 0.5012135911240411|
|    643|0.49978723662136165|
+-------+-------------------+
only showing top 20 rows



In [445]:
r_df.show(2)

+-------+-------+------+---------+
|user_id|film_id|rating|       hz|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
+-------+-------+------+---------+
only showing top 2 rows



In [518]:
prt_df = spark.sql( 'SELECT T1.film_id, T1.user_id, T1.rating, T2.users_mean_rating, T1.rating - T2.users_mean_rating as r_ ' + 
           ' FROM ' +
           ' (SELECT film_id, user_id, rating FROM ratings  WHERE user_id IN (SELECT user_id FROM ngb_30)) T1, ' +
           ' (SELECT user_id, users_mean_rating FROM rf) T2 ' +
           '  WHERE T1.user_id = T2.user_id ' 
         )


In [519]:
prt_df.registerTempTable('prt')

In [533]:
prt2_df = spark.sql( 'SELECT T1.film_id, T1.user_id, T1.r_, T2.sau,  T1.r_ * T2.sau as for_sum ' +
           ' FROM ' +
           ' (SELECT film_id, user_id, r_ FROM prt) T1, ' +
           ' (SELECT user_id, sau FROM ngb_30) T2 ' +
           ' WHERE T1.user_id = T2.user_id ' 
)

In [534]:
prt2_df.registerTempTable('prt2')

In [535]:
prt2_df.show()

+-------+-------+-------------------+------------------+--------------------+
|film_id|user_id|                 r_|               sau|             for_sum|
+-------+-------+-------------------+------------------+--------------------+
|    787|    591|-0.6547999999999998|0.5140159694936672|-0.33657765682445323|
|   1017|    591|-0.6547999999999998|0.5140159694936672|-0.33657765682445323|
|   1099|    591| 1.3452000000000002|0.5140159694936672|  0.6914542821628813|
|    191|    591| 1.3452000000000002|0.5140159694936672|  0.6914542821628813|
|    382|    591| 0.3452000000000002|0.5140159694936672| 0.17743831266921403|
|    322|    591|-1.6547999999999998|0.5140159694936672| -0.8505936263181204|
|    740|    591| 0.3452000000000002|0.5140159694936672| 0.17743831266921403|
|    204|    591| 0.3452000000000002|0.5140159694936672| 0.17743831266921403|
|    286|    591| 0.3452000000000002|0.5140159694936672| 0.17743831266921403|
|     56|    591| 0.3452000000000002|0.5140159694936672| 0.17743

In [552]:
prt3_df = spark.sql('SELECT film_id, sum(for_sum) as chislit, ' +
          ' sum(CASE WHEN sau < 0 THEN -1 * sau ELSE sau END) as znam, ' + 
          ' sum(for_sum) / sum(CASE WHEN sau < 0 THEN -1 * sau ELSE sau END) as res_'
          ' FROM prt2 GROUP BY film_id')

In [553]:
prt3_df.registerTempTable('prt3')

In [563]:
spark.sql('SELECT film_id FROM prt3 ' +
          ' WHERE film_id IN (SELECT DISTINCT(film_id) FROM ratings WHERE user_id != 94)' + 
          ' ORDER BY res_ DESC').collect()

[Row(film_id='1367'),
 Row(film_id='1529'),
 Row(film_id='513'),
 Row(film_id='1099'),
 Row(film_id='1103'),
 Row(film_id='1080'),
 Row(film_id='493'),
 Row(film_id='647'),
 Row(film_id='320'),
 Row(film_id='902'),
 Row(film_id='127'),
 Row(film_id='134'),
 Row(film_id='1021'),
 Row(film_id='100'),
 Row(film_id='603'),
 Row(film_id='610'),
 Row(film_id='185'),
 Row(film_id='474'),
 Row(film_id='854'),
 Row(film_id='1143'),
 Row(film_id='1514'),
 Row(film_id='1022'),
 Row(film_id='285'),
 Row(film_id='180'),
 Row(film_id='654'),
 Row(film_id='518'),
 Row(film_id='718'),
 Row(film_id='694'),
 Row(film_id='882'),
 Row(film_id='483'),
 Row(film_id='494'),
 Row(film_id='919'),
 Row(film_id='524'),
 Row(film_id='1203'),
 Row(film_id='262'),
 Row(film_id='313'),
 Row(film_id='192'),
 Row(film_id='641'),
 Row(film_id='64'),
 Row(film_id='246'),
 Row(film_id='169'),
 Row(film_id='56'),
 Row(film_id='23'),
 Row(film_id='644'),
 Row(film_id='272'),
 Row(film_id='648'),
 Row(film_id='357'),
 Row(f

In [564]:
result = {
    'average_film_ratings' : 59.453, 
    'average_user_ratings' : 106.0445, 
    'completeness' : 0.063,
    'pearson_neighbours' : [92, 303 , 115 , 49 , 645 , 747 ,457 ,537 ,135 ,933 ,643 ,
                            292 ,339 ,389 ,591 ,880 ,343 ,
                            269 ,595 ,592 ,560 ,773 ,624 ,916 ,854 ,249 ,267 ,293 ,360 ,
                            323 ], 
    'pearson_top10' : [1367, 1529, 513, 1099, 1103, 1080, 493, 647, 320, 902] 
}   

In [565]:
import json
with open('lab08.json', 'w') as file:
    json.dump(result, file)

In [194]:
spark.sql('SELECT user_id, rating, film_id FROM ratings WHERE user_id = 94 ')

DataFrame[user_id: string, rating: string, film_id: string]

In [179]:
spark.sql('SELECT user_id, rating, film_id FROM ratings WHERE user_id = {}'.format('296'))

DataFrame[user_id: string, rating: string, film_id: string]

In [215]:
spark.sql('SELECT rating FROM ratings ' + 
          ' WHERE user_id = {} and film_id in (SELECT film_id FROM ratings WHERE user_id = 94)'.format('296')).show()

+------+
|rating|
+------+
|     5|
|     5|
|     4|
|     4|
|     4|
|     5|
|     5|
|     4|
|     5|
|     4|
|     2|
|     5|
|     4|
|     5|
|     5|
|     5|
|     4|
|     5|
|     4|
|     5|
+------+
only showing top 20 rows



In [327]:
curr_users_mean_rating = spark.sql('SELECT users_mean_rating FROM user_user WHERE user_id = {}'.format('467')).take(1)[0].users_mean_rating
my_users_mean_rating = spark.sql('SELECT users_mean_rating FROM user_user WHERE user_id = {}'.format('94')).take(1)[0].users_mean_rating
print(curr_users_mean_rating, my_users_mean_rating)

3.6818 3.6575


In [285]:
spark.sql(
'SELECT  T2.user_id, T1.rating as rating_94, T2.rating as rating_u, T1.film_id,' + 
' int(T1.rating) - {} as myR_,  int(T2.rating) - {} as uR_,'.format(my_users_mean_rating, curr_users_mean_rating) +
' (int(T1.rating) - {}) * (int(T2.rating) - {}) as myR_POW_uR_,'.format(my_users_mean_rating, curr_users_mean_rating) +    
' POW(int(T1.rating) - {}, 2) as myR_2,'.format(my_users_mean_rating) + 
' POW(int(T2.rating) - {}, 2) as uR_2'.format(curr_users_mean_rating) +    
' FROM ' +
'(SELECT user_id, rating, film_id FROM ratings WHERE user_id = 94) T1, ' +
'(SELECT user_id, rating, film_id FROM ratings WHERE user_id = {}) T2 '.format('296') +
' WHERE T1.film_id = T2.film_id'
).count()


76

In [324]:
spark.sql(
'SELECT ' +
#'    sum(myR_POW_uR_) as sum_myR_POW_uR_,'     +
#'       CASE WHEN count(T3.user_id)/50 < 1 THEN count(T3.user_id)/50 ELSE 1 END as correction, ' +    
'       (sum(myR_POW_uR_) / (sqrt(sum(myR_2)) * sqrt(sum(uR_2))))  * (CASE WHEN count(T3.user_id)/50 < 1 THEN count(T3.user_id)/50 ELSE 1 END) as corr_s_a_u, '    
#'       sum(myR_2) as sum_myR_2, '                +
#'       sum(uR_2) as sum_uR_2, '                  +
#'       count(T3.user_id)/50 as count_del_50, '     +
#'       sqrt(sum(myR_2)) as sqrt_sum_myR_2, '     +   
#'       sqrt(sum(uR_2))  as sqrt_ssum_uR_2, '     + 
#'       sqrt(sum(myR_2)) * sqrt(sum(uR_2))  as Pow_sqrt, '  +
'       CASE WHEN sum(myR_2) = 0 OR sum(uR_2) = 0 THEN 0 ELSE sum(myR_POW_uR_) / (sqrt(sum(myR_2)) * sqrt(sum(uR_2))) END as s_a_u ' +
    ' FROM (' +
' SELECT T2.user_id as user_id, T1.rating as rating_94, T2.rating as rating_u, T1.film_id,' + 
' int(T1.rating) - {} as myR_,  int(T2.rating) - {} as uR_,'.format(my_users_mean_rating, curr_users_mean_rating) +
' (int(T1.rating) - {}) * (int(T2.rating) - {}) as myR_POW_uR_,'.format(my_users_mean_rating, curr_users_mean_rating) +    
' POW(int(T1.rating) - {}, 2) as myR_2,'.format(my_users_mean_rating) + 
' POW(int(T2.rating) - {}, 2) as uR_2'.format(curr_users_mean_rating) +    
' FROM ' +
'(SELECT user_id, rating, film_id FROM ratings WHERE user_id = 94) T1, ' +
'(SELECT user_id, rating, film_id FROM ratings WHERE user_id = {}) T2 '.format('467') +
' WHERE T1.film_id = T2.film_id' +
    ') T3 GROUP BY user_id'
    
).take(1)[0].corr_s_a_u










0.01659367754126844